In [64]:
#imports
import pandas as pd
import numpy as np
from lxml import etree
import os
from bs4 import BeautifulSoup
import requests
import xml.etree.ElementTree as ET
import validators
import datetime
import re
import string
import nltk
from nltk.corpus import stopwords

from collections import Counter

In [3]:
#downloads
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('rslp')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\caperei\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\caperei\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\caperei\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\caperei\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
#parameters
characters_for_name=60
legislature='dar/01'
cycle='14'

stop_words_nltk = nltk.corpus.stopwords.words('portuguese')
#stop_words = ["the","it","she","he", "a"] #Uncomment this line if you want to use your own list of stopwords.
#The stemmers and lemmers need to be initialized before bing run
#porter = nltk.stem.porter.PorterStemmer()
snowball = nltk.stem.snowball.SnowballStemmer('portuguese')
#wordnet = nltk.stem.WordNetLemmatizer()
#RSLP=nltk.stem.RSLPStemmer()

In [5]:
#utils

def add_zeros(int_):
    if len(str(int_))<2:
        return '00'+str(int_)
    if len(str(int_))<3:
        return '0'+str(int_)
    if len(str(int_))==3:
        return str(int_)

def get_text_from_html(url__):
    from bs4 import BeautifulSoup
    res = requests.get(url__)
    html_page = res.content
    soup = BeautifulSoup(html_page, 'html.parser')
    return str(soup.noscript)

def read_input_table(current_directory="C:\\Users\caperei",cycle='14',session='1' ):

    data_directory=f"\portuguese_open_data\data\cycle\{cycle}\session\{session}\\numbers_dates_pages.csv"
    input_data=pd.read_csv(r"C:\Users\caperei\portuguese_open_data\data\cyle\14\session\1\numbers_dates_pages.csv")
    return input_data

In [6]:
#regex
def check_dialog(text_):
    pat="(?:</p><p>.*?\(.*?\): —)"
    match=re.findall(pat, text_)
    if match is None :
        return 'no_dialog'
    else:
        for m in match:
            
            return m

def extract_dialog(full_text,characters_for_name):    
    if full_text is None:
        return None
    else:
        dialogs=[(m.start(0), m.end(0)) for m in re.finditer('</p><p></p><p>(.+?): —', full_text)]
        out=[]
        if len(dialogs)==1:
            out.append(full_text[dialogs[0][1]-characters_for_name:])
        if len(dialogs)>1:
            for i in range(0, len(dialogs)-1):
                out.append(full_text[dialogs[i][1]-characters_for_name:dialogs[i+1][0]])
            out.append(full_text[dialogs[len(dialogs)-1][1]-50:])
        return out

    
def extract_party_name(dialogs):
    if dialogs is None:
        return None
    else:
        res=[]
        for i in dialogs:
                    positions=[(m.start(0), m.end(0)) for m in re.finditer('\((.+?)\): —', i)]
                    if len(positions)==0:
                        res.append(['No','No'])
                    else:
                        party=re.findall('\((.+?)\)',i)
                        name_aux=i[positions[0][1]-50:positions[0][1]]
                        name=re.sub(r'\b\w{1,2}\b', '', name_aux).replace('.','').replace ('()','').replace('  ','').replace('<','').replace('>','').replace('/','').replace(': —','')
                        if len(party)>0:
                            party=party[0]
                        res.append([party,name])
        return res

def add_speech_next_page(df):
    for pi in range(1,df.page.max()+1):
        if len(df[df.page==pi].speech.values[0])>0:
            speeches=df[df.page==pi].speech.values[0]
            if '</noscript>' in speeches[-1]:
                
                for n in range(1,df.page.max()-pi):
                    if ': —' in df[df.page==pi+n].text_1.values[0]:
                        
                        in_=df[df.page==pi+n].text_1.values[0].find(': —')
                        df[df.page==pi].speech.values[0][-1]=df[df.page==pi].speech.values[0][-1]+' '+str(df[df.page==pi+n].text_1.values[0][0:in_])
                        
                        break
                    else:
                        
                        df[df.page==pi].speech.values[0][-1]=df[df.page==pi].speech.values[0][-1]+' '+str(df[df.page==pi+n].text_1.values[0])
    return df


#defining the function to remove punctuation
def remove_punctuation(text):
  if(type(text)==float):
    return text
  ans=""  
  for i in text:     
    if i not in string.punctuation+'<p></p>ºª':
      ans+=i    
  return ans

def normlizeTokens(tokenLst, stopwordLst = None, stemmer = None):
    #We can use a generator here as we just need to iterate over it

    #Lowering the case and removing non-words
    workingIter = (w.lower() for w in tokenLst if w.isalpha())

    #Now we can use the semmer, if provided
    if stemmer is not None:
        workingIter = (stemmer.stem(w) for w in workingIter)
         
    #And remove the stopwords
    if stopwordLst is not None:
        workingIter = (w for w in workingIter if w not in stopwordLst)
    #We will return a list with the stopwords removed
    return list(workingIter)

def generate_N_grams(text,ngram=1):
  words=[word for word in text]  
  temp=zip(*[words[i:] for i in range(0,ngram)])
  ans=[' '.join(ngram) for ngram in temp]
  return ans

In [18]:
#functions scrapping
def build_transcripts_table_from_html(session=['01'],legislature='dar/01', cycle='14'):
    df = pd.DataFrame(columns=['legislature','cycle','session','number','date','page', 'text_1','url'])        
    #session=['01'] #['01','02','03']
    numbers_dates=list(zip(input_data.number, input_data.date,input_data.pages )) #[0:1]
    pages=input_data.pages.values #[0:1]
    s_c=[]
    n_c=[]
    d_c=[]
    p_c=[]
    u_c=[]
    t_c=[]
    for s in session:
        for number, date, page in numbers_dates:
            number=add_zeros(number)
            date=datetime.datetime.strptime(date, '%d/%m/%Y').strftime('%Y-%m-%d')
            for page_i in range(1,page+1):
                    url_=f'https://debates.parlamento.pt/catalogo/r3/{legislature}/{cycle}/{s}/{number}/{date}/{page_i}'
                    if validators.url(url_):
                        #print (url_)
                        u_c.append(url_)
                        p_c.append(page_i)
                        t_c.append(get_text_from_html(url_))
                        s_c.append(s)
                        n_c.append(number)
                        d_c.append(date)
    df['page']=p_c
    df['session']=s_c
    df['number']=n_c
    df['date']=d_c

    df['text_1']=t_c
    df['url']=u_c
    df['legislature']=legislature
    df['cycle']=cycle    
    return df

def add_party_speaker(df,characters_for_name ):
    #add columns
    df['speech']=[extract_dialog(t,characters_for_name) for t in df.text_1 ]
    df['party']=[extract_party_name(t) for t in df.speech ]
    df=add_speech_next_page(df)
    #re arrange by speaker and party
    df1=df.explode(['speech', 'party']).reset_index(drop=True)
    df2=df1.dropna(subset=['party'])
    df2['party_s']=[t[0] for t in df2.party]
    df2['speaker']=[t[1] for t in df2.party]
    df2=df2[df2.party_s!='No']
    return df2

In [8]:
input_data=read_input_table()
input_data.head(3)

,number,date,pages
0,1,25/10/2019,13
1,2,30/10/2019,124
2,3,31/10/2019,26


In [9]:
#=build_transcripts_table_from_html()
#df.to_pickle('portuguese_transcripts_s1.pkl') 
df = pd.read_pickle('portuguese_transcripts_s1.pkl')

In [10]:
df.head()

,legislature,cycle,session,number,date,page,text_1,url,speech,party
0,dar/01,14,01,001,2019-10-25,1,"<noscript>\n<p>Sábado, 26 de outubro de 2019 ...",https://debates.parlamento.pt/catalogo/r3/dar/...,[],[]
1,dar/01,14,01,001,2019-10-25,2,<noscript>\n<p>I SÉRIE — NÚMERO 1 </p><p></p><...,https://debates.parlamento.pt/catalogo/r3/dar/...,[es.</p><p></p><p>A Sr.ª Ana Catarina Mendonça...,"[[PS, Ana Catarina Mendonça Mendes ], [No, No]..."
2,dar/01,14,01,001,2019-10-25,3,<noscript>\n<p>26 DE OUTUBRO DE 2019 </p><p>3 ...,https://debates.parlamento.pt/catalogo/r3/dar/...,"[er votado, Sr. Presidente. </p><p></p><p>O Sr...","[[No, No], [PS, Pedro Delgado Alves ]]"
3,dar/01,14,01,001,2019-10-25,4,<noscript>\n<p>I SÉRIE — NÚMERO 1 </p><p></p><...,https://debates.parlamento.pt/catalogo/r3/dar/...,[],[]
4,dar/01,14,01,001,2019-10-25,5,<noscript>\n<p>26 DE OUTUBRO DE 2019 </p><p>5 ...,https://debates.parlamento.pt/catalogo/r3/dar/...,"[residente, muito obrigado. </p><p></p><p>O Sr...","[[No, No]]"


In [19]:
df2=add_party_speaker(df,characters_for_name )

<ipython-input-18-99982d4d3f5c>:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['party_s']=[t[0] for t in df2.party]
<ipython-input-18-99982d4d3f5c>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['speaker']=[t[1] for t in df2.party]


In [20]:
df2['speech1']= df2['speech'].apply(lambda x:remove_punctuation(x))
df2['tokenized_text'] = df2['speech1'].apply(lambda x: nltk.word_tokenize(x))
df2['normalized_tokens'] = df2['tokenized_text'].apply(lambda x: normlizeTokens(x, stopwordLst = stop_words_nltk, stemmer = snowball))
df2['normalized_tokens_count'] = df2['normalized_tokens'].apply(lambda x: len(x))
df2['uni_grams'] = df2['normalized_tokens'].apply(lambda x: generate_N_grams(x,1))
df2['bi_grams'] = df2['normalized_tokens'].apply(lambda x: generate_N_grams(x,2))
df2['tri_grams'] = df2['normalized_tokens'].apply(lambda x: generate_N_grams(x,3))

In [23]:
#df2.party_s.unique()
parties=['PS', 'PSD', 'BE', 'PCP', 'CDS-PP', 'PAN', 'PEV','CH','IL','L','CDS','PCP']
df3=df2[df2.party_s.isin(parties)]

In [28]:
df3.head(3)

,legislature,cycle,session,number,date,page,text_1,url,speech,party,speech1,tokenized_text,normalized_tokens,normalized_tokens_count,uni_grams,bi_grams,tri_grams,party_s,speaker
1,dar/01,14,01,001,2019-10-25,2,<noscript>\n<p>I SÉRIE — NÚMERO 1 </p><p></p><...,https://debates.parlamento.pt/catalogo/r3/dar/...,es.</p><p></p><p>A Sr.ª Ana Catarina Mendonça ...,"[PS, Ana Catarina Mendonça Mendes ]",esA Sr Ana Catarina Mendonça Mendes PS — Sras ...,"[esA, Sr, Ana, Catarina, Mendonça, Mendes, PS,...","[esa, sr, ana, catarin, mendonc, mend, ps, sra...",90,"[esa, sr, ana, catarin, mendonc, mend, ps, sra...","[esa sr, sr ana, ana catarin, catarin mendonc,...","[esa sr ana, sr ana catarin, ana catarin mendo...",PS,Ana Catarina Mendonça Mendes
5,dar/01,14,01,001,2019-10-25,3,<noscript>\n<p>26 DE OUTUBRO DE 2019 </p><p>3 ...,https://debates.parlamento.pt/catalogo/r3/dar/...,o. </p><p></p><p>O Sr. Pedro Delgado Alves (PS...,"[PS, Pedro Delgado Alves ]",o O Sr Pedro Delgado Alves PS — Sr Presidente ...,"[o, O, Sr, Pedro, Delgado, Alves, PS, —, Sr, P...","[sr, pedr, delg, alves, ps, sr, president, sra...",682,"[sr, pedr, delg, alves, ps, sr, president, sra...","[sr pedr, pedr delg, delg alves, alves ps, ps ...","[sr pedr delg, pedr delg alves, delg alves ps,...",PS,Pedro Delgado Alves
12,dar/01,14,01,001,2019-10-25,8,<noscript>\n<p>I SÉRIE — NÚMERO 1 </p><p></p><...,https://debates.parlamento.pt/catalogo/r3/dar/...,</p><p>A Sr.ª Ana Catarina Mendonça Mendes (PS...,"[PS, Ana Catarina Mendonça Mendes ]",A Sr Ana Catarina Mendonça Mendes PS — Sr Pres...,"[A, Sr, Ana, Catarina, Mendonça, Mendes, PS, —...","[sr, ana, catarin, mendonc, mend, ps, sr, pres...",271,"[sr, ana, catarin, mendonc, mend, ps, sr, pres...","[sr ana, ana catarin, catarin mendonc, mendonc...","[sr ana catarin, ana catarin mendonc, catarin ...",PS,Ana Catarina Mendonça Mendes


In [27]:

r_df3=df3[df3.party_s.isin(right)]
l_df3=df3[df3.party_s.isin(left)]

In [29]:
df3[df3.party_s.isin(right)].uni_grams

14       [grã, sr, fern, negrã, psd, sr, president, hoj...
20       [meirel, sr, cecíl, meirel, cdspp, sr, preside...
21            [ssõ, sr, joã, pinh, alme, cdspp, muit, bem]
22       [sr, cecíl, meirel, cdspp, cad, vez, fiz, form...
40       [cdspp, ch, sr, adã, silv, psd, agor, vã, chov...
                               ...                        
21984         [sr, joã, pinh, alme, cdspp, senhor, govern]
21986      [ps, sr, joã, pinh, alme, cdspp, tenh, vergonh]
21989    [sab, sr, joã, pinh, alme, cdspp, convers, jog...
21991    [dem, sr, joã, pinh, alme, cdspp, convers, jog...
21993           [sr, joã, pinh, alme, cdspp, nã, vai, ter]
Name: uni_grams, Length: 5111, dtype: object

In [104]:
right=[ 'PSD',  'CDS-PP', 'CH','IL','CDS']
left=[ 'PS', 'BE', 'PCP', 'PAN', 'PEV','L','PCP']

def create_frequency_table_grams(n_gram=1, input=df3, right_parties=right, left_parties=left):
    grams_d={1: 'uni_grams', 2: 'bi_grams', 3: 'tri_grams'}
    r_grams=[item for sublist in df3[df3.party_s.isin(right)][grams_d[n_gram]] for item in sublist]
    l_grams=[item for sublist in df3[df3.party_s.isin(left)][grams_d[n_gram]] for item in sublist]

    total_counter = Counter([item for sublist in df3[grams_d[n_gram]] for item in sublist])
    right_counter = Counter(r_grams)
    left_counter = Counter(l_grams)

    df_all = pd.DataFrame.from_dict(total_counter, orient='index').reset_index()
    df_all.columns=['phrase','count']
    df_all['f_right']=[right_counter[x] for x in df_all.phrase]
    df_all['f_left']=[left_counter[x] for x in df_all.phrase]

    df_all['f_right_minus']=[f_all_except(right_counter, x) for x in df_all.phrase]
    df_all['f_left_minus']=[f_all_except(left_counter, x) for x in df_all.phrase]
    df_all['f_left_total']=get_total_party(left_counter)
    df_all['f_right_total']=get_total_party(right_counter)

    df_all['f_right_norm']=df_all['f_right']/df_all['f_right_total']
    df_all['f_left_norm']=df_all['f_left']/df_all['f_left_total']

    df_all['f_right_minus_norm']=df_all['f_right_minus']/df_all['f_right_total']
    df_all['f_left_minus_norm']=df_all['f_left_minus']/df_all['f_left_total']

    return df_all


def calculate_pearson(df_all):
    aa=df_all['f_right_norm']*df_all['f_left_minus_norm'] 
    bb=df_all['f_left_norm']*df_all['f_right_minus_norm']
    cc=aa-bb
    dd=cc*cc
    d11=df_all['f_right_norm']+df_all['f_left_norm']
    d22=df_all['f_right_norm']+df_all['f_right_minus_norm']
    d33=df_all['f_left_norm']+df_all['f_left_minus_norm']
    d44=df_all['f_right_minus_norm']+df_all['f_left_minus_norm']
    denom=d11*d22*d33*d44
    pp=dd/denom
    return pp

In [105]:
dfg=create_frequency_table_grams(n_gram=1, input=df3, right_parties=right, left_parties=left)


In [107]:
dfg2=create_frequency_table_grams(n_gram=2, input=df3, right_parties=right, left_parties=left)

In [ ]:
dfg3=create_frequency_table_grams(n_gram=3, input=df3, right_parties=right, left_parties=left)

In [ ]:
dfg2.to_pickle('dfg2.pkl') 
dfg3.to_pickle('dfg3.pkl') 

In [ ]:
dfg['pearson_quad']=calculate_pearson(dfg)
dfg2['pearson_quad']=calculate_pearson(dfg2)
dfg3['pearson_quad']=calculate_pearson(dfg3)

In [106]:
dfg

,phrase,count,f_right,f_left,f_right_minus,f_left_minus,f_left_total,f_right_total,f_right_norm,f_left_norm,f_right_minus_norm,f_left_minus_norm
0,esa,7,2,5,268384,463452,463457,268386,0.000007,0.000011,0.999993,0.999989
1,sr,24874,10587,14287,257799,449170,463457,268386,0.039447,0.030827,0.960553,0.969173
2,ana,590,285,305,268101,463152,463457,268386,0.001062,0.000658,0.998938,0.999342
3,catarin,317,63,254,268323,463203,463457,268386,0.000235,0.000548,0.999765,0.999452
4,mendonc,116,5,111,268381,463346,463457,268386,0.000019,0.000240,0.999981,0.999760
...,...,...,...,...,...,...,...,...,...,...,...,...
14988,serrã,1,0,1,268386,463456,463457,268386,0.000000,0.000002,1.000000,0.999998
14989,demorous,1,0,1,268386,463456,463457,268386,0.000000,0.000002,1.000000,0.999998
14990,desidrat,1,0,1,268386,463456,463457,268386,0.000000,0.000002,1.000000,0.999998
14991,conhecems,1,0,1,268386,463456,463457,268386,0.000000,0.000002,1.000000,0.999998


In [ ]:
df_all['pearson_quad']=calculate_pearson(df_all)
df_all.sort_values(by=['pearson_quad'], ascending=False)

In [93]:
grams_d={1: 'uno_grams', 2: 'bi_grams', 3: 'tri_grams'}

In [100]:
#[[grams_d[2]]]

df3[grams_d[2]]

,bi_grams
1,"[esa sr, sr ana, ana catarin, catarin mendonc,..."
5,"[sr pedr, pedr delg, delg alves, alves ps, ps ..."
12,"[sr ana, ana catarin, catarin mendonc, mendonc..."
14,"[grã sr, sr fern, fern negrã, negrã psd, psd s..."
16,"[lie soar, soar sr, sr pedr, pedr fili, fili s..."
...,...
21984,"[sr joã, joã pinh, pinh alme, alme cdspp, cdsp..."
21986,"[ps sr, sr joã, joã pinh, pinh alme, alme cdsp..."
21989,"[sab sr, sr joã, joã pinh, pinh alme, alme cds..."
21991,"[dem sr, sr joã, joã pinh, pinh alme, alme cds..."


In [45]:


df_all = pd.DataFrame.from_dict(total_counter, orient='index').reset_index()
df_all.columns=['phrase','count']

In [62]:
df_all['f_right']=[right_counter[x] for x in df_all.phrase]
df_all['f_left']=[left_counter[x] for x in df_all.phrase]

In [68]:
def f_all_except(counter_r_l, excp):
    exclude_keys = [excp]
    ans = [counter_r_l[k] for k in set(list(counter_r_l.keys())) - set(exclude_keys)]
    return np.sum(ans)

def get_total_party(counter_r_l):
    ans = [counter_r_l[k] for k in list(counter_r_l.keys())]
    return np.sum(ans)

In [71]:
df_all['f_right_minus']=[f_all_except(right_counter, x) for x in df_all.phrase]
df_all['f_left_minus']=[f_all_except(left_counter, x) for x in df_all.phrase]
df_all['f_left_total']=get_total_party(left_counter)
df_all['f_right_total']=get_total_party(right_counter)

df_all['f_right_norm']=df_all['f_right']/df_all['f_right_total']
df_all['f_left_norm']=df_all['f_left']/df_all['f_left_total']

df_all['f_right_minus_norm']=df_all['f_right_minus']/df_all['f_right_total']
df_all['f_left_minus_norm']=df_all['f_left_minus']/df_all['f_left_total']

In [88]:
def calculate_pearson(df_all):
    aa=df_all['f_right_norm']*df_all['f_left_minus_norm'] 
    bb=df_all['f_left_norm']*df_all['f_right_minus_norm']
    cc=aa-bb
    dd=cc*cc
    d11=df_all['f_right_norm']+df_all['f_left_norm']
    d22=df_all['f_right_norm']+df_all['f_right_minus_norm']
    d33=df_all['f_left_norm']+df_all['f_left_minus_norm']
    d44=df_all['f_right_minus_norm']+df_all['f_left_minus_norm']
    denom=d11*d22*d33*d44
    pp=dd/denom
    return pp

In [89]:
df_all['pearson_quad']=calculate_pearson(df_all)
df_all.sort_values(by=['pearson_quad'], ascending=False)

,phrase,count,f_right,f_left,f_right_minus,f_left_minus,f_left_total,f_right_total,f_right_norm,f_left_norm,f_right_minus_norm,f_left_minus_norm,pearson_quad
107,psd,4893,3708,1185,264678,462272,463457,268386,0.013816,0.002557,0.986184,0.997443,3.903199e-03
644,cdspp,1782,1644,138,266742,463319,463457,268386,0.006126,0.000298,0.993874,0.999702,2.652234e-03
585,pcp,3381,292,3089,268094,460368,463457,268386,0.001088,0.006665,0.998912,0.993335,2.013744e-03
6,ps,3913,466,3447,267920,460010,463457,268386,0.001736,0.007438,0.998264,0.992562,1.779745e-03
111,be,1973,82,1891,268304,461566,463457,268386,0.000306,0.004080,0.999694,0.995920,1.627947e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1086,concert,71,26,45,268360,463412,463457,268386,0.000097,0.000097,0.999903,0.999903,1.258987e-10
3251,distânc,101,37,64,268349,463393,463457,268386,0.000138,0.000138,0.999862,0.999862,9.711563e-11
1221,comromet,109,40,69,268346,463388,463457,268386,0.000149,0.000149,0.999851,0.999851,4.187400e-11
893,sobr,2599,953,1646,267433,461811,463457,268386,0.003551,0.003552,0.996449,0.996448,3.603611e-11


In [55]:
[x for x in df_all.phrase]

['esa',
 'sr',
 'ana',
 'catarin',
 'mendonc',
 'mend',
 'ps',
 'sras',
 'srs',
 'deut',
 'nom',
 'boa',
 'rax',
 'arlament',
 'send',
 'gru',
 'maioritári',
 'nest',
 'assembl',
 'reúblic',
 'cabem',
 'inic',
 'trabalh',
 'dest',
 'nov',
 'legislatur',
 'mesm',
 'gost',
 'rimeir',
 'lug',
 'saud',
 'tod',
 'eleit',
 'ara',
 'cumr',
 'aquel',
 'reseit',
 'intuiçã',
 'democrát',
 'cad',
 'exerc',
 'mandat',
 'reresent',
 'daquel',
 'eleg',
 'aqu',
 'estar',
 'parlament',
 'part',
 'social',
 'muit',
 'honr',
 'design',
 'resid',
 'noss',
 'sessã',
 'eduard',
 'ferr',
 'rodrigu',
 'rest',
 'president',
 'cessant',
 'eço',
 'sub',
 'pedr',
 'delg',
 'alves',
 'asso',
 'entã',
 'leitur',
 'relatóri',
 'arec',
 'comissã',
 'eventual',
 'verific',
 'pod',
 'seguint',
 'teor',
 'dias',
 'mês',
 'outubr',
 'hor',
 'minut',
 'reun',
 'sal',
 'paláci',
 'sã',
 'bent',
 'roced',
 'noscrit',
 'i',
 'séri',
 'númer',
 'oder',
 'dia',
 'xiv',
 'reuniã',
 'deliber',
 'mes',
 'constituíd',
 'elo',
 'f

In [51]:
df_all

,phrase,count
0,esa,7
1,sr,24874
2,ana,590
3,catarin,317
4,mendonc,116
...,...,...
14988,serrã,1
14989,demorous,1
14990,desidrat,1
14991,conhecems,1
